In [ ]:
%matplotlib widget
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from bmcs_shear.dic_crack import\
    DICGrid, DICInpUnstructuredPoints, DICStateFields, DICAlignedGrid
from bmcs_shear.dic_crack.dic_crack_list import DICCrackList
import numpy as np
np.seterr(divide ='ignore', invalid='ignore');
from bmcs_shear.dic_crack import sz_tests_series_2023

In [ ]:
from cgitb import small


def validate_crack_tips(X_Ca, h_x):
    # Calculate the distances between neighboring points
    x_C, y_C = X_Ca.T
    dists = np.diff(x_C)

    # 
    if not np.all(dists > 0):
        raise ValueError("The horizontal positions of cracks" + 
                         "must be monotonically increasing.")

    return X_Ca

In [ ]:
test = sz_tests_series_2023.B9_TV1
dcl = sz_tests_series_2023.new_dcl(test)

In [ ]:
dcl.dsf.dic_grid.t = 0.15
X_tip_t_Ca = dcl.X_tip_t_Ca

In [ ]:
x_range, y_range = X_tip_t_Ca.T
x_0, y_0, x_1, y_1 = dcl.dsf.dic_grid.X_frame
y_range

In [ ]:
x_0_range = np.hstack([[x_0], x_range, [x_1]])
y_0_range = np.hstack([y_range[:1], y_range, y_range[-1:]])
x_1_range = np.copy(x_0_range)
y_1_range = np.ones_like(x_1_range) * y_1
y_0_range

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,3))
ax.scatter(x_0_range, y_0_range)
ax.scatter(x_1_range, y_1_range)
ax.set_aspect('equal')

In [ ]:
import ibvpy.api as ib

In [ ]:
n_E, n_F = x_0_range.size, 1
xmodel = ib.XDomainFEGrid(coord_min=(x_0, x_1), coord_max=(y_0, y_1),
                 integ_factor=1, shape=(n_E, n_F),  # number of elements!
                 fets=ib.FETS2D4Q());

In [ ]:
xmodel.mesh.n_grid_elems

In [ ]:
x_IJ, y_IJ = np.einsum('IJa->aIJ', dcl.dsf.dic_grid.X_IJa)
n_T, n_I, n_J = dcl.dsf.dic_grid.n_T, dcl.dsf.dic_grid.n_I, dcl.dsf.dic_grid.n_J

In [ ]:
U_IJa = dcl.dsf.dic_grid.U_IJa

# Set spacing in x and y directions
dx = x_IJ[1,0]-x_IJ[0,0]
dy = y_IJ[0,1]-y_IJ[0,0]

# Compute the gradients
gradient_x = np.gradient(U_IJa, dx, edge_order=2, axis=0)  # Gradient with respect to x
gradient_y = np.gradient(U_IJa, dy, edge_order=2, axis=1)  # Gradient with respect to y

# Now we have the gradients of the displacement field with respect to x and y.
# We can compute the tensor F as follows:
F_IJab = np.empty((n_I, n_J, 2, 2))

F_IJab[..., 0, 0] = gradient_x[..., 0]
F_IJab[..., 0, 1] = gradient_y[..., 0]
F_IJab[..., 1, 0] = gradient_x[..., 1]
F_IJab[..., 1, 1] = gradient_y[..., 1]

# Compute the strain tensor using numpy.einsum
eps_IJab = 0.5 * (F_IJab + np.einsum('...ij->...ji', F_IJab))

In [ ]:
U_TIJa = dcl.dsf.dic_grid.U_TIJa

# Set spacing in x and y directions
dx = x_IJ[1,0]-x_IJ[0,0]
dy = y_IJ[0,1]-y_IJ[0,0]

# Compute the gradients
gradient_x = np.gradient(U_TIJa, dx, edge_order=2, axis=1)  # Gradient with respect to x
gradient_y = np.gradient(U_TIJa, dy, edge_order=2, axis=2)  # Gradient with respect to y

In [ ]:
# Now we have the gradients of the displacement field with respect to x and y.
# We can compute the tensor F as follows:
F_TIJab = np.empty((n_T, n_I, n_J, 2, 2))

F_TIJab[..., 0, 0] = gradient_x[..., 0]
F_TIJab[..., 0, 1] = gradient_y[..., 0]
F_TIJab[..., 1, 0] = gradient_x[..., 1]
F_TIJab[..., 1, 1] = gradient_y[..., 1]

# Compute the strain tensor using numpy.einsum
eps_TIJab = 0.5 * (F_TIJab + np.einsum('...ij->...ji', F_TIJab))

In [ ]:
x_MN, y_MN = dcl.dsf.xy_MN
#eps_MNab = dcl.dsf.get_z_MN_ironed(x_IJ, y_IJ, eps_TIJab[-1], 15, x_MN, y_MN)
eps_MNab = dcl.dsf.get_z_MN_ironed(x_IJ, y_IJ, eps_IJab, 15, x_MN, y_MN)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import Normalize
from matplotlib.colorbar import ColorbarBase

def plot_eps_field(x_, y_, eps_):

    fig, ax = plt.subplots(figsize=(10,4))

    pos = np.ma.masked_less(eps_, 0)
    neg = np.ma.masked_greater(eps_, 0)

    cmap_pos = plt.cm.Reds
    cmap_neg = plt.cm.Blues_r

    levels_pos = np.linspace(0, pos.max(), 10)
    levels_neg = np.linspace(neg.min(), 0, 10)

    cf_pos = ax.contourf(x_, y_, pos, levels_pos, cmap=cmap_pos)
    cf_neg = ax.contourf(x_, y_, neg, levels_neg, cmap=cmap_neg)

    cax_neg = fig.add_axes([0.1, 0.1, 0.4, 0.05])
    cax_pos = fig.add_axes([0.5, 0.1, 0.4, 0.05])

    ColorbarBase(cax_pos, cmap=cmap_pos, norm=Normalize(vmin=0, vmax=pos.max()), orientation='horizontal')
    ColorbarBase(cax_neg, cmap=cmap_neg, norm=Normalize(vmin=neg.min(), vmax=0), orientation='horizontal')

    levels = [0]
    fmt = {0: '0'}
    cs = ax.contour(x_, y_, eps_, levels, linewidths=1, colors='k')
    ax.clabel(cs, fmt=fmt, inline=1)
    ax.set_aspect('equal')


In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,3))
fig.canvas.header_visible=False
y_J = y_IJ[0, :]
eps_meso_IJab = eps_IJab[0, :, 0, 0]
#eps_meso_IJab = eps_TIJab[-1, 0, :, 0, 0]
ax.plot(eps_meso_IJab, y_J, 'o-');
y_M = y_MN[0, :]
eps_macro_IJab = eps_MNab[0, :, 0, 0]
ax.plot(eps_macro_IJab, y_M, 'o-');

In [ ]:
eps_IJa, _ = np.linalg.eig(eps_IJab)
eps_indices = np.argmax(np.fabs(eps_IJa), axis=-1)
minmax_eps_IJ = np.take_along_axis(eps_IJa, eps_indices[..., np.newaxis], axis=-1).squeeze(axis=-1)
plot_eps_field(x_IJ, y_IJ, minmax_eps_IJ)

In [ ]:

eps_MNa, V_eps_MNab = np.linalg.eig(eps_MNab)
eps_indices = np.argmax(np.fabs(eps_MNa), axis=-1)
minmax_eps_MN = np.take_along_axis(eps_MNa, eps_indices[..., np.newaxis], axis=-1).squeeze(axis=-1)
#plot_eps_field(x_MN, y_MN, minmax_eps_MN)
plot_eps_field(x_MN, y_MN, eps_MNab[...,0,0])

In [ ]:
V_eps_MNa = np.take_along_axis(V_eps_MNab, eps_indices[..., np.newaxis, np.newaxis], axis=-2).squeeze(axis=-2)
u_eps_MNa, v_eps_MNa = np.einsum('MNa->aMN', V_eps_MNa)
fig, ax = plt.subplots()
scale = 0.05
ax.quiver(x_MN, y_MN, u_eps_MNa, v_eps_MNa, angles='xy', scale_units='xy', scale=scale)
ax.quiver(x_MN, y_MN, -u_eps_MNa, -v_eps_MNa, angles='xy', scale_units='xy', scale=scale)
ax.set_aspect('equal')

In [ ]:
fig, ax = plt.subplots()
scale = 0.05
U_eps_MNa, V_eps_MNa = np.einsum('MNab->bMNa', V_eps_MNab)
u_eps_MN, v_eps_MN = np.einsum('MNa->aMN', U_eps_MNa)
ax.quiver(x_MN, y_MN, u_eps_MN, v_eps_MN, angles='xy', scale_units='xy', scale=scale)
ax.quiver(x_MN, y_MN, -u_eps_MN, -v_eps_MN, angles='xy', scale_units='xy', scale=scale)

u_eps_MN, v_eps_MN = np.einsum('MNa->aMN', V_eps_MNa)
ax.quiver(x_MN, y_MN, u_eps_MN, v_eps_MN, angles='xy', scale_units='xy', scale=scale)
ax.quiver(x_MN, y_MN, -u_eps_MN, -v_eps_MN, angles='xy', scale_units='xy', scale=scale)

ax.set_aspect('equal')